# Experiment 1

### Environment Setup

In [1]:
%%bash -s

# set root folder and get environment
export root="/home/fabric/work"
source $root/env
pwd

/home/fabric/work


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fabric-fss-utils 1.2.2 requires Jinja2==2.11.3, but you have jinja2 3.1.2 which is incompatible.
fabric-fss-utils 1.2.2 requires MarkupSafe==1.1.1, but you have markupsafe 2.1.1 which is incompatible.


In [2]:
# imports
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from fabrictestbed_extensions.fablib.node import Node
from threading import Thread
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import uuid
from collections import defaultdict

# show fablib config
fablib = fablib_manager()             
fablib.show_config()

# constants
root = "/home/fabric/work"

# component names
slice_name = "experiment-1"
server_site, client_site = fablib.get_random_sites(count=2, avoid=['AL2S','STAR', 'MAX'])
print(f"{server_site=}, {client_site=}")

# switch detail
client_switch_name = "client-switch"
server_switch_name = "server-switch"
switch_cores = 2
switch_ram = 8
switch_disk = 100

# host details
client_host_name = "client-host"
server_host_name = "server-host"
image = 'default_ubuntu_20'
server_host_subnet=IPv4Network('10.0.1.0/24')
server_host_addr=IPv4Address('10.0.1.1')
client_host_subnet=IPv4Network('10.0.2.0/24')
client_host_addr=IPv4Address('10.0.2.2')

# network details
net_server_switch_name = "net-server-switch"
net_client_switch_name = "net-client-switch"
net_switch_switch_name = "net-switch-switch"

-----------------------------------  ---------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           6ce270de-788d-4e07-8bae-3206860a6387
bastion_username                     brmclemo_0000026977
bastion_key_filename                 /home/fabric/work/ssh/fabric-bastion
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/ssh/fabric-sliver.pub
slice_private_key_file               /home/fabric/work/ssh/fabric-sliver
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  ---------------------------------------
server_site='UTAH', client_site='DALL'


In [3]:
# some helper functions
__id_counter = defaultdict(int)
def id(name=None):
  global __id_counter
  __id_counter[name] += 1
  return f"{name}-{str(__id_counter[name]).zfill(4)}"

## Setup Network

In [4]:

# create the slice
slice = fablib.new_slice(name=slice_name)

# add server switch
server_switch = slice.add_node(name=server_switch_name, site=server_site, image=image)
server_switch_iface_host = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
server_switch_iface_client_switch = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add client switch
client_switch = slice.add_node(name=client_switch_name, site=client_site, image=image)
client_switch_iface_host = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
client_switch_iface_server_switch = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add server host
server_host = slice.add_node(name=server_host_name, site=server_site, image=image)
server_host_iface = server_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add client host
client_host = slice.add_node(name=client_host_name, site=client_site, image=image)
client_host_iface = client_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add networks
server_switch_net = slice.add_l2network(name=net_server_switch_name, interfaces=[server_switch_iface_host, server_host_iface])
client_switch_net = slice.add_l2network(name=net_client_switch_name, interfaces=[client_switch_iface_host, client_host_iface])
switch_switch_net = slice.add_l2network(name=net_switch_switch_name, interfaces=[server_switch_iface_client_switch, client_switch_iface_server_switch])

# submit slice for creation
slice.submit()



-----------  ------------------------------------
Slice Name   experiment-1
Slice ID     f70fd147-94a7-45bb-b219-844666d1b51c
Slice State  StableOK
Lease End    2022-11-14 16:48:55 +0000
-----------  ------------------------------------

Retry: 13, Time: 192 sec

ID                                    Name           Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  -------------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
84cd86e5-092b-4f5a-af82-7973fd738d6b  server-switch  UTAH    utah-w5.fabric-testbed.net        2      8      10  default_ubuntu_20  2001:1948:417:7:f816:3eff:fef3:835d     Active
d29d4bed-3075-4396-a0cb-ce82fd167506  client-switch  DALL    dall-w1.fabric-testbed.net        2      8      10  default_ubuntu_20  2001:400:a100:3000:f816:3eff:fe30:cedd  Active
0

'f70fd147-94a7-45bb-b219-844666d1b51c'

In [5]:
# set up nodes
ip_addrs = dict()
slice = fablib.get_slice(name=slice_name)
l2_available_addrs = list(l2_subnet := IPv4Network("192.168.2.0/24"))[1:]
l3_available_addrs = list(l3_subnet := IPv4Network("192.168.3.0/24"))[1:]
node_addrs = defaultdict(lambda: l2_available_addrs.pop(0))
for node in slice.get_nodes():
  node_name = node.get_name()
  print(f"{node_name=}")

  # setup
  node.execute("sudo apt update && sudo apt install net-tools && pip install netifaces")
  node.upload_directory('code','~')

  # give ip to all nodes
  for iface in node.get_interfaces():
    iface_name = iface.get_name()
    print(f"{iface_name=}")
    iface.ip_addr_add(addr=node_addrs[node_name], subnet=l2_subnet)
    iface.ip_link_up()
    
  if 'host' in node_name:
    node.ip_route_add(subnet=l3_subnet, gateway=node_addrs[f"{node_name[:-5]}-switch"])


node_name='server-switch'
iface_name='server-switch-nic-0001-p1'
iface_name='server-switch-nic-0002-p1'
node_name='client-switch'
iface_name='client-switch-nic-0003-p1'
iface_name='client-switch-nic-0004-p1'
node_name='server-host'
iface_name='server-host-nic-0005-p1'
node_name='client-host'
iface_name='client-host-nic-0006-p1'


In [6]:
# install and start server switch
raise
slice = fablib.get_slice(name=slice_name)
for net in slice.get_l2networks():
  net.show()

AttributeError: 'NetworkService' object has no attribute 'show'

## Configure Network Components

In [ ]:
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]
f"{list(subnet)[0]}"

### Configure Hosts

In [ ]:
raise
# config information
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/"

# configure server host
server_host = slice.get_node(name=server_host_name)        
if type(ip_address(server_host.get_management_ip())) is IPv6Address:
    stdout, stderr = server_host.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    print(server_host_name)
    print(f"stdout:\n{stdout}\n")
    print(f"stderr:\n{stderr}\n")
server_host_os_iface = server_host.get_interface(network_name=net_server_switch_name)
server_host_os_iface.ip_addr_add(addr=server_host_addr, subnet=server_host_subnet)
stdout, stderr = server_host.execute(host_config_script)
print(server_host_name)
print(f"stdout:\n{stdout}\n")
print(f"stderr:\n{stderr}\n")

# configure client host
client_host = slice.get_node(name=client_host_name)        
if type(ip_address(client_host.get_management_ip())) is IPv6Address:
    stdout, stderr = client_host.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    print(client_host_name)
    print(f"stdout:\n{stdout}\n")
    print(f"stderr:\n{stderr}\n")
client_host_os_iface = client_host.get_interface(network_name=net_client_switch_name)
client_host_os_iface.ip_addr_add(addr=client_host_addr, subnet=client_host_subnet)
stdout, stderr = client_host.execute(host_config_script)
print(client_host_name)
print(f"stdout:\n{stdout}\n")
print(f"stderr:\n{stderr}\n")


### Configure Switches

In [ ]:
server_switch = slice.get_node(name=server_switch_name)
server_switch_os_iface = server_switch.get_interface(network_name=net_server_switch_name)
switch_switch_os_iface = server_switch.get_interface(network_name=net_switch_switch_name)
    
if type(ip_address(server_switch.get_management_ip())) is IPv4Address:
    management_ip_type = "IPv4"
else:
    management_ip_type = "IPv6"

file_attributes = server_switch.upload_file(f'{root}/P4_bmv2/scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {server_switch_os_iface.get_os_interface()} {switch_switch_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
s1_config_thread = server_switch.execute_thread(command)
config_threads[server_switch] = s1_config_thread

In [ ]:
client_switch = slice.get_node(name=client_switch_name)
client_switch_os_iface = client_switch.get_interface(network_name=net_client_switch_name)
switch_switch_os_iface = client_switch.get_interface(network_name=net_switch_switch_name)
    
if type(ip_address(client_switch.get_management_ip())) is IPv4Address:
    management_ip_type = "IPv4"
else:
    management_ip_type = "IPv6"

file_attributes = client_switch.upload_file(f'{root}/P4_bmv2/scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {server_switch_os_iface.get_os_interface()} {switch_switch_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
s1_config_thread = client_switch.execute_thread(command)
config_threads[client_switch] = s1_config_thread

In [ ]:
# print the output of all nodes
for node, thread in config_threads.items():
  stdout, stderr = thread.result()
  print(node.get_username())
  print(f"Config thread node {node.get_name()} complete")
  print(f"stdout:\n{stdout}")
  print(f"stderr:\n{stderr}")

## Delete Slice

In [ ]:
raise
slice = fablib.get_slice(name=slice_name)
slice.delete()